In [19]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import tensorflow
from tensorflow_model_optimization.python.core.keras.compat import keras
import tensorflow_model_optimization as tfmot
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
df = pd.read_csv(r"C:\Users\vodna\OneDrive\Desktop\inno\DL\compresed_model\data.csv")
df.drop(columns = ["Timestamp","Email Address"],axis=1,inplace=True)

In [3]:
le = LabelEncoder()
for column in df.columns:
    le = le.fit(df[column])
    df[column] = le.fit_transform(df[column])

In [4]:
obj = KMeans(n_clusters=2,n_init=5).fit(df)
clusters = obj.labels_
df["labels"] = clusters

In [5]:
data = df.drop(columns="labels",axis=1)
labels = df["labels"] 

# Normal Model

In [6]:
start = time.time()
normal_model = keras.models.load_model(r"C:\Users\vodna\OneDrive\Desktop\inno\DL\compresed_model\normal.h5")
end = time.time()
print(f"time taken to load a normal model is {end-start:.4f} seconds")


time taken to load a normal model is 0.3999 seconds


In [7]:
len(data)

149

In [8]:
a=int(input('Enter number of prediction'))
c = 0
time_taken = 0
predicted_labels = []
tot_loss=0
for index in range(len(data)):
    if c == a:
        break
    observation = data.loc[index]
    observation = np.array(observation).reshape(1,-1)
    start = time.time()
    prediction = normal_model.predict(observation,verbose=0)
    end = time.time()
    t = end - start
    time_taken += t
    c += 1
    if prediction[0][0] < 0.5:
        predicted_labels.append(0)
    else:
        predicted_labels.append(1)
    
   
    loss = -(labels[index]*np.log(prediction[0][0]) + (1-labels[index])*np.log(1-prediction[0][0]))
    tot_loss += loss
    
predicted_labels = np.array(predicted_labels)
true_labels = np.array(labels.loc[0:a-1])
test_accuracy = (predicted_labels == true_labels).mean()



print(f"time taken to make a {a} prediction using normal_model is {time_taken:.4f} seconds")
print(f"accuracy for {a} prediction using normal_model is {test_accuracy:.4f}")
print(f"loss for {a} prediction using normal_model is {tot_loss/a:.4f} ")


time taken to make a 149 prediction using normal_model is 10.1792 seconds
accuracy for 149 prediction using normal_model is 0.9597
loss for 149 prediction using normal_model is 0.2070 


# Compressed Model

In [9]:
start = time.time()
compressed_model = tensorflow.lite.Interpreter(model_path=r"C:\Users\vodna\OneDrive\Desktop\inno\DL\compresed_model\normal_compressed.tflite")
end = time.time()
print(f"time taken to load a compressed model is {end-start:.4f} seconds")


time taken to load a compressed model is 0.0022 seconds


c:\Users\vodna\anaconda3\envs\compressed_model\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [10]:
# allocate tensors to the model
compressed_model.allocate_tensors()

In [11]:
input_tensor = compressed_model.get_input_details() #--> how to pass the data to the model
output_tensor = compressed_model.get_output_details() #--> how to calucate the output from the model


In [13]:
a=int(input('Enter number of prediction'))
c = 0
time_taken=0
predicted_labels = []
tot_loss=0
for index in range(len(data)):
    if c == a:
        break
    observation = data.loc[index].astype(np.float32)
    observation = np.array(observation).reshape(1,-1)
    start = time.time()
    compressed_model.invoke() 
    compressed_model.set_tensor(input_tensor[0]["index"],observation)
    output = compressed_model.get_tensor(output_tensor[0]["index"])
    end = time.time()
    t=end-start
    time_taken+=t
    if output[0][0] < 0.5:
        predicted_labels.append(0)
    else:
        predicted_labels.append(1)
    loss = -(labels[index]*np.log(output[0][0]) + (1-labels[index])*np.log(1-output[0][0]))
    tot_loss += loss
    c +=1
predicted_labels = np.array(predicted_labels)
true_labels = np.array(labels.loc[0:a-1])
test_accuracy = (predicted_labels == true_labels)



print(f"time taken to make a 100 prediction using compressed_model is {time_taken:.4f} seconds")
print(f"accuracy for 100 prediction using compressed_model is {test_accuracy.mean():.4f} ")
print(f"loss for 100 prediction using compressed_model is {tot_loss/a:.4f} ")

time taken to make a 100 prediction using compressed_model is 0.0000 seconds
accuracy for 100 prediction using compressed_model is 0.5168 
loss for 100 prediction using compressed_model is 1.1449 


# Compresssed + Quantized Model

In [14]:
start = time.time()
quantized_model = tensorflow.lite.Interpreter(model_path=r"C:\Users\vodna\OneDrive\Desktop\inno\DL\compresed_model\normal_quantized_compressed.tflite") 
end = time.time()
print(f"time taken to load a compressed model is {end-start:.4f} seconds")


time taken to load a compressed model is 0.0026 seconds


c:\Users\vodna\anaconda3\envs\compressed_model\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [15]:
# allocate tensors to the model
quantized_model.allocate_tensors()

In [16]:
input_tensor = quantized_model.get_input_details() #--> how to pass the data to the model
output_tensor = quantized_model.get_output_details() #--> how to calucate the output from the model


In [17]:
a=int(input('Enter number of prediction'))
c = 0
time_taken=0
predicted_labels = []
tot_loss=0
for index in range(len(data)):
    if c == a:
        break
    observation = data.loc[index].astype(np.float32)
    observation = np.array(observation).reshape(1,-1)
    start = time.time()
    quantized_model.invoke() 
    quantized_model.set_tensor(input_tensor[0]["index"],observation) 
    output = quantized_model.get_tensor(output_tensor[0]["index"])
    end = time.time()
    t=end-start
    time_taken+=t
    if output[0][0] < 0.5:
        predicted_labels.append(0)
    else:
        predicted_labels.append(1)
    loss = -(labels[index]*np.log(output[0][0]) + (1-labels[index])*np.log(1-output[0][0]))
    tot_loss += loss
    c +=1
predicted_labels = np.array(predicted_labels)
true_labels = np.array(labels.loc[0:a-1])
test_accuracy = (predicted_labels == true_labels)



print(f"time taken to make a {a} prediction using quantized_model is {time_taken:.4f} seconds")
print(f"accuracy for {a} prediction using quantized_model is {test_accuracy.mean():.4f} ")
print(f"loss for {a} prediction using quantized_model is {tot_loss/a:.4f} ")

time taken to make a 149 prediction using quantized_model is 0.0010 seconds
accuracy for 149 prediction using quantized_model is 0.6107 
loss for 149 prediction using quantized_model is 0.6830 
